In [23]:
import pandas as pd
df=pd.read_csv("outputs/test10@gmail.com.csv")

In [24]:
df.head(5)

,PMID,Title,PublicationDate,NonAcademicAuthors,CompanyAffiliations,CorrespondingAuthorEmail
0,39136969,"This differentiation block will not stand, man...",13-8-2024,NaN,"Department of Pharmacy, University of Californ...",NaN
1,39136895,A lower initial dose of bosutinib for patients...,20-4-2024,NaN,"Department of Pharmacokinetics, Akita Universi...",naotot@doc.med.akita-u.ac.jp.
2,39136784,Analysis of the shorter drug survival times fo...,31-3-2024,"Peter Bartz-Bazzanella, Patrick Wurth, Wolfgan...","Hannover Medical School, Institute for Clinica...",Strunz_P@ukw.de.
3,39135965,Impact of Antifungal Prophylaxis Continuation ...,27-2-2024,"Jana Dickter, Sanjeet Singh Dadwal, Deepa Nana...","Department of Pharmacy, City of Hope National ...",NaN
4,39135125,Systematic literature review of the somatic co...,25-4-2023,"Jack Said, Sheun Oshinbolu, Gillian C Sibbring...","BioMarin Pharmaceutical Inc., 105 Digital Driv...",Kaleigh.Whitehall@bmrn.com.
